Adjustment Reconciliation Tool (Notebook Version with Enhanced Uploads)

In [60]:
import pandas as pd
from datetime import datetime
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import Output
import base64
import io
import matplotlib.pyplot as plt
import seaborn as sns


# Set Seaborn style for beautiful plots
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (10, 6) # Default figure size

In [58]:
# --- Date formats
DATE_FORMATS = [
    '%Y-%m-%d', '%Y/%m/%d', '%d.%m.%Y', '%Y.%m.%d',
    '%d/%m/%Y', '%-d/%-m/%Y', '%-d.%-m.%Y'
]

In [3]:
# Date for FX rate (can be dynamically set or user input)
# This will be used for any FX conversions
fx_rate_date = '2025-07-11' # YYYY-MM-DD for consistency

# Various Date Formats to handle different date representations in CSVs
date_formats = [
    '%Y-%m-%d',
    '%Y/%m/%d',
    '%d.%m.%Y',
    '%Y.%m.%d',
    '%d/%m/%Y',
    '%-d/%-m/%Y',
    '%-d.%-m.%Y'
]

# Fuzzy matching threshold for bank names (0-100)
FUZZY_MATCH_THRESHOLD = 80

In [4]:
# --- Bank name normalization maps
BANK_NAME_MAP = {
    'central bank of kenya': 'cbk', 'kenya commercial bank': 'kcb',
    'kingdom bank': 'kingdom', 'absa bank': 'absa', 'ABSA Bank': 'absa',
    'equity bank': 'equity', 'i&m bank': 'i&m', 'ncba bank kenya plc': 'ncba', 'ncba bank': 'ncba'
}

In [5]:
FOREIGN_REPLACEMENTS = {
    'ncba bank kenya plc': 'ncba', 'ncba bank': 'ncba', 'equity bank': 'equity',
    'i&m bank': 'i&m', 'central bank of kenya': 'cbk', 'kenya commercial bank': 'kcb',
    'kcb bank': 'kcb', 'sbm bank (kenya) limited': 'sbm', 'sbm bank': 'sbm'
}


fx_upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='Upload FX Tracker'
)
fx_output = Output()
fx_sheet_dropdown = widgets.Dropdown(description='Sheet:', layout=widgets.Layout(width='300px'), visible=False)
fx_column_selector = widgets.SelectMultiple(description='Columns:', layout=widgets.Layout(width='300px'))
fx_column_renames = {}
fx_column_rename_box = widgets.VBox()
fx_file_label = widgets.Label(value="No FX file uploaded.")
process_fx_btn = widgets.Button(description='Process FX Data', button_style='success')
clear_fx_btn = widgets.Button(description='Clear FX Upload', button_style='danger')
fx_controls = widgets.HBox([fx_upload_widget, clear_fx_btn, process_fx_btn])

fx_raw_file = None
fx_trade_df = pd.DataFrame()
fx_sheet_names = []

# ========== HELPERS ==========

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

def generate_download_link(df, filename):
    csv_data = df.to_csv(index=False)
    b64 = base64.b64encode(csv_data.encode()).decode()
    return widgets.HTML(value=f'<a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">Download {filename}</a>')

def extract_excel_sheets(file_dict):
    excel_file = pd.ExcelFile(io.BytesIO(file_dict['content']))
    return excel_file.sheet_names

def build_column_rename_fields(columns):
    fields = []
    fx_column_renames.clear()
    for col in columns:
        input_widget = widgets.Text(value=col, description=col, layout=widgets.Layout(width='400px'))
        fx_column_renames[col] = input_widget
        fields.append(input_widget)
    fx_column_rename_box.children = fields

# ========== CALLBACKS ==========

@fx_output.capture()
def load_fx_file(change):
    global fx_raw_file, fx_sheet_names
    fx_output.clear_output()
    fx_column_selector.options = []
    fx_column_rename_box.children = []
    fx_sheet_dropdown.options = []
    fx_sheet_dropdown.visible = False
    fx_raw_file = None
    files = fx_upload_widget.value

    if not files:
        fx_file_label.value = "No FX file uploaded."
        return

    file = files[0]
    fx_file_label.value = f"Uploaded: {file['name']}"
    fx_raw_file = file

    if file['name'].endswith('.xlsx'):
        fx_sheet_names = extract_excel_sheets(file)
        fx_sheet_dropdown.options = fx_sheet_names
        fx_sheet_dropdown.value = fx_sheet_names[0]
        fx_sheet_dropdown.visible = True
        display(widgets.HTML("<b>Select sheet before processing</b>"))
    else:
        # CSV case – preview immediately
        df = pd.read_csv(io.BytesIO(file['content']))
        fx_column_selector.options = list(df.columns)
        build_column_rename_fields(df.columns)
        display(df.head())

def process_fx_data(change):
    global fx_trade_df
    fx_output.clear_output()
    if not fx_raw_file:
        print("⚠️ No FX file loaded.")
        return

    try:
        if fx_raw_file['name'].endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(fx_raw_file['content']), sheet_name=fx_sheet_dropdown.value)
        else:
            df = pd.read_csv(io.BytesIO(fx_raw_file['content']))
        
        df.columns = df.columns.str.strip()
        selected_cols = list(fx_column_selector.value)
        if selected_cols:
            df = df[selected_cols]

        renamed_cols = {col: w.value for col, w in fx_column_renames.items() if col in df.columns and w.value}
        df.rename(columns=renamed_cols, inplace=True)

        fx_trade_df = df

        print("✅ FX Data Processed:")
        display(fx_trade_df.head())
        display(generate_download_link(fx_trade_df, "processed_fx_data.csv"))

    except Exception as e:
        print(f"❌ Error processing FX file: {e}")

def clear_fx_data(change):
    global fx_raw_file, fx_trade_df
    fx_upload_widget.value = ()
    fx_file_label.value = "No FX file uploaded."
    fx_column_selector.options = []
    fx_column_rename_box.children = []
    fx_output.clear_output()
    fx_raw_file = None
    fx_trade_df = pd.DataFrame()

# ========== EVENTS ==========

fx_upload_widget.observe(load_fx_file, names='value')
clear_fx_btn.on_click(clear_fx_data)
process_fx_btn.on_click(process_fx_data)
fx_column_selector.observe(lambda change: build_column_rename_fields(change['new']), names='value')
# Handle sheet selection (only needed for Excel)
def handle_sheet_selection(change):
    if not fx_raw_file or not fx_raw_file['name'].endswith('.xlsx'):
        return
    try:
        df = pd.read_excel(io.BytesIO(fx_raw_file['content']), sheet_name=change['new'])
        df.columns = df.columns.str.strip()
        fx_column_selector.options = list(df.columns)
        build_column_rename_fields(df.columns)
        display(df.head())
    except Exception as e:
        fx_output.clear_output()
        print(f"❌ Failed to load sheet: {e}")

fx_sheet_dropdown.observe(handle_sheet_selection, names='value')

# ========== DISPLAY UI ==========

print("📥 FX Upload with Sheet Selector + Column Mapping")
display(fx_controls, fx_file_label, fx_sheet_dropdown, fx_column_selector, fx_column_rename_box, fx_output)

📥 FX Upload with Sheet Selector + Column Mapping


Label(value='No FX file uploaded.')

Dropdown(description='Sheet:', layout=Layout(width='300px'), options=(), value=None)

SelectMultiple(description='Columns:', layout=Layout(width='300px'), options=(), value=())

VBox()

Output()

,Request ID,Intermediary Account,Payment Channel,Counterparty Bank,Counterparty Account ID,Counterparty Name,Counterparty Sub-account,Transfer Reference No.,Transaction Narrative,TX ID,Customer Account Number,Account Name,Account Channel,Operation,Product,Currency,Amount,Status,Requested At,Completed At
0,d000105eaa68f918010003967,Equity Bank,NaN,Choice Microfinance Bank,1,CELESTINE MUGAMBI KIBAARA,NaN,CELESTINE MUGAMBI KIBAARA JULY 2025 CASH ADVANCE,CELESTINE MUGAMBI KIBAARA JULY 2025 CASH ADVA...,CTRANS0213eb866c802001,46010000015565,CELESTINE MUGAMBI KIBAARA,Choice Personal,debit,OP0024 - C - Account Adjustment - Credit Balance,KES,4654080,Successful,2025-07-11 17:49:20,2025-07-11 17:49:53
1,d000105ea9f1e624010000c0b,Unclaimed Funds,RTGS,Equity Bank Limited,0260285969384,QUEBELL LLC LIMITED,NaN,C777100725125118,QUEBELL LLC LIMITED RTGS TRANSFER,B0028168588193666457608,46011000004559,QUEBELL LLC LIMITED,Choice Merchant,credit,OP0031 - B - Unclaimed Funds - Credit,KES,5172000,Successful,2025-07-11 17:16:49,2025-07-11 17:21:54
2,d000105ea9c3373c010001907,ABSA Bank,RTGS,Absa Bank Kenya PLC,2051000164,Amatta Enterprises Limited,NaN,NaN,Amatta Enterprises Limited RTGS TRANSFER,UTRANS0213e96757b02000,46013000992957,Amatta Enterprises Limited,Koa,credit,TP0040 - BaaS B - Transfer In - RTGS & EFT,KES,300000,Successful,2025-07-11 17:04:05,2025-07-11 17:12:49
3,d000105ea9b696e0010009455,Central Bank of Kenya,RTGS,I&M Bank Limited,00202801381850,TRACKING HUB LIMITED,NaN,000004106468,Pesi advance remittance Jephiter Momanyi,UTRANS0213e96d20402000,46013000916360,SWITCHLINK AFRICA LTD,Switchlink,credit,TP0040 - BaaS B - Transfer In - RTGS & EFT,KES,15900,Successful,2025-07-11 17:00:38,2025-07-11 17:13:12
4,d000105ea977fb6c01000d2fd,Central Bank of Kenya,RTGS,ECO Bank Limited,6682001803,LAMI TECHNOLOGIES LIMITED,NaN,P50RTGO251920107,LOTUSTRANSFER,UTRANS0213e7dfe3e02000,46013000000686,Lami Technologies Limited,Koa,credit,TP0040 - BaaS B - Transfer In - RTGS & EFT,KES,500000,Successful,2025-07-11 16:43:32,2025-07-11 16:46:05


In [28]:
# === Imports ===
import io
import base64
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [27]:
# === Globals ===
bank_raw_files = []
bank_dfs = {}

# File table widget
uploaded_file_table = widgets.VBox()

In [29]:
# === Helpers ===
def normalize_bank_key(name):
    return name.strip().lower().replace(' ', '_').replace('.csv', '').replace('.xlsx', '')

def generate_download_link(df, filename):
    csv_data = df.to_csv(index=False)
    b64 = base64.b64encode(csv_data.encode()).decode()
    return widgets.HTML(value=f'<a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">Download {filename}</a>')

def extract_excel_sheets(file_dict):
    excel_file = pd.ExcelFile(io.BytesIO(file_dict['content']))
    return excel_file.sheet_names

def build_bank_file_ui(file_dict):
    name = file_dict['name']
    content = file_dict['content']
    file_size_kb = len(content) / 1024
    file_box = widgets.VBox()
    file_label = widgets.HTML(f"<b>🧾 {name}</b> <span style='color:gray'>({file_size_kb:.1f} KB)</span>")
    
    dropdown = widgets.Dropdown(description="Sheet:", visible=False, layout=widgets.Layout(width="300px"))
    column_selector = widgets.SelectMultiple(description="Columns:", layout=widgets.Layout(width="300px"))
    rename_fields_box = widgets.VBox()
    rename_fields = {}

    preview_output = widgets.Output(layout={'border': '1px solid lightgray', 'padding': '5px'})
    df = None

    if name.endswith('.xlsx'):
        try:
            sheet_names = extract_excel_sheets(file_dict)
            dropdown.options = sheet_names
            dropdown.value = sheet_names[0]
            dropdown.visible = True
        except Exception as e:
            file_label.value += f" ❌ <span style='color:red'>Error reading Excel sheets: {e}</span>"

    def try_read_csv(bytes_obj):
        encodings = ['utf-8', 'latin1', 'ISO-8859-1']
        for enc in encodings:
            try:
                return pd.read_csv(io.BytesIO(bytes_obj), encoding=enc)
            except Exception:
                continue
        raise ValueError("Failed to decode CSV using common encodings.")

    try:
        if name.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(content), sheet_name=dropdown.value)
        else:
            df = try_read_csv(content)

        df.columns = df.columns.str.strip()
        column_selector.options = list(df.columns)

        for col in df.columns:
            input_widget = widgets.Text(value=col, description=col, layout=widgets.Layout(width='400px'))
            rename_fields[col] = input_widget
        rename_fields_box.children = list(rename_fields.values())

        with preview_output:
            print("📊 Preview:")
            display(df.head())

    except Exception as e:
        file_label.value += f" ❌ <span style='color:red'>Error reading file: {e}</span>"

    bank_raw_files.append({
        'file_dict': file_dict,
        'dropdown': dropdown,
        'column_selector': column_selector,
        'rename_fields': rename_fields_box,
        'rename_map': rename_fields,
        'df_preview': df,
        'key': normalize_bank_key(name)
    })

    file_box.children = [file_label, dropdown, column_selector, rename_fields_box, preview_output]
    return file_box

In [ ]:
# Widgets
bank_upload_widget = widgets.FileUpload(accept='.csv,.xlsx', multiple=True, description='Upload Bank Statement(s)')
clear_bank_btn = widgets.Button(description='Clear Bank Uploads', button_style='danger')
process_bank_btn = widgets.Button(description='Process Bank Data', button_style='success')
bank_file_label = widgets.Label(value="No bank files uploaded.")
bank_output = widgets.Output()
bank_file_boxes = widgets.VBox()

# Controls group
bank_controls = widgets.HBox([bank_upload_widget, clear_bank_btn, process_bank_btn])

In [31]:
def load_bank_files(change):
    bank_output.clear_output()
    bank_file_boxes.children = []
    bank_raw_files.clear()

    files = bank_upload_widget.value

    # Normalize file input
    normalized_files = []
    if isinstance(files, dict):  # Classic Notebook
        for name, meta in files.items():
            normalized_files.append({
                'name': name,
                'content': meta['content']
            })
    elif isinstance(files, (list, tuple)):  # JupyterLab/Colab
        for file_obj in files:
            normalized_files.append({
                'name': file_obj['name'],
                'content': file_obj['content']
            })

    if normalized_files:
        uploaded_file_table.children = []  # Reset table
        header_row = widgets.HBox([
            widgets.HTML("<b>File</b>", layout=widgets.Layout(width="40%")),
            widgets.HTML("<b>Size (KB)</b>", layout=widgets.Layout(width="15%")),
            widgets.HTML("<b>Type</b>", layout=widgets.Layout(width="15%")),
            widgets.HTML("<b>Action</b>", layout=widgets.Layout(width="20%"))
        ])
        table_rows = [header_row]
        names = []

        for file_dict in normalized_files:
            box = build_bank_file_ui(file_dict)
            bank_file_boxes.children += (box,)
            name = file_dict['name']
            content = file_dict['content']
            size_kb = len(content) / 1024
            file_ext = name.split('.')[-1].lower()
            file_icon = "📊" if file_ext == "xlsx" else "📄"
            file_type = "Excel" if file_ext == "xlsx" else "CSV"
            names.append(name)

            remove_btn = widgets.Button(
                description="Remove",
                button_style="danger",
                layout=widgets.Layout(width="90px", height="30px")
            )

            def make_remove_callback(name_to_remove):
                def _remove(_):
                    bank_file_boxes.children = tuple(
                        box for box in bank_file_boxes.children
                        if not any(name_to_remove in str(child) for child in box.children)
                    )
                    bank_raw_files[:] = [
                        entry for entry in bank_raw_files if entry['file_dict']['name'] != name_to_remove
                    ]
                    uploaded_file_table.children = tuple(
                        row for row in uploaded_file_table.children
                        if name_to_remove not in str(row.children[0].value)
                    )
                    if not bank_raw_files:
                        bank_file_label.value = "No bank files uploaded."
                        uploaded_file_table.children = [header_row]
                return _remove

            remove_btn.on_click(make_remove_callback(name))

            row = widgets.HBox([
                widgets.HTML(f"{file_icon} {name}", layout=widgets.Layout(width="40%")),
                widgets.HTML(f"{size_kb:.1f}", layout=widgets.Layout(width="15%")),
                widgets.HTML(file_type, layout=widgets.Layout(width="15%")),
                remove_btn
            ])
            table_rows.append(row)

        uploaded_file_table.children = table_rows
        bank_file_label.value = f"Uploaded: {', '.join(names)}"
    else:
        bank_file_label.value = "No bank files uploaded."
        uploaded_file_table.children = []

In [11]:
def clear_bank_files(change):
    global bank_raw_files, bank_dfs
    bank_upload_widget.value = ()
    bank_upload_widget._counter = 0
    bank_raw_files.clear()
    bank_dfs.clear()
    bank_file_label.value = "No bank files uploaded."
    uploaded_file_table.children = []
    bank_file_boxes.children = []
    bank_output.clear_output()

In [32]:
def process_bank_files(change):
    global bank_dfs
    bank_output.clear_output()
    if not bank_raw_files:
        with bank_output:
            print("⚠️ No bank files to process.")
            return

    for entry in bank_raw_files:
        file = entry['file_dict']
        sheet = entry['dropdown'].value if file['name'].endswith('.xlsx') else None
        selected_cols = list(entry['column_selector'].value)
        rename_map = {col: widget.value for col, widget in entry['rename_map'].items() if widget.value}

        try:
            if file['name'].endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(file['content']), sheet_name=sheet)
            else:
                df = pd.read_csv(io.BytesIO(file['content']))

            df.columns = df.columns.str.strip()
            if selected_cols:
                df = df[selected_cols]
            df.rename(columns=rename_map, inplace=True)
            key = entry['key']
            bank_dfs[key] = df

            with bank_output:
                print(f"✅ Processed {file['name']}:")
                display(df.head())
                display(generate_download_link(df, f"{key}_processed.csv"))
        except Exception as e:
            with bank_output:
                print(f"❌ Error processing {file['name']}: {e}")

In [33]:
# Register event handlers
bank_upload_widget.observe(load_bank_files, names='value')
clear_bank_btn.on_click(clear_bank_files)
process_bank_btn.on_click(process_bank_files)

In [34]:
# Display UI
print("🏦 Enhanced Bank Upload Interface:")
display(bank_controls, bank_file_label, uploaded_file_table, bank_file_boxes, bank_output)

🏦 Enhanced Bank Upload Interface:


Label(value='No bank files uploaded.')

VBox()

VBox()

Output()

In [35]:
# === Dropdown for mode selection ===
mode_selector = widgets.Dropdown(
    options=['local', 'foreign'],
    value='local',
    description='Mode:',
)
display(mode_selector)

Dropdown(description='Mode:', options=('local', 'foreign'), value='local')

In [37]:
def normalize_bank_key_local(bank_name, currency):
    bank_code = BANK_NAME_MAP.get(bank_name.lower().strip())
    if not bank_code:
        return None
    return f"{bank_code} {currency.lower()}"

In [38]:
def normalize_bank_key_foreign(intermediary):
    parts = intermediary.split('-')
    if len(parts) < 2:
        return None
    bank_name = parts[0].strip().lower()
    currency = parts[1].strip().lower()
    for k, v in FOREIGN_REPLACEMENTS.items():
        if bank_name.startswith(k):
            bank_name = bank_name.replace(k, v)
            break
    return f"{bank_name} {currency}"

In [39]:

def generate_download_link(df, filename):
    """Generates a download link for a DataFrame."""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:text/csv;base64,{b64}" download="{filename}">Download {filename}</a>'
    return HTML(href)

def safe_float(x):
    """Safely converts a value to a float, handling commas, non-numeric inputs, and ensuring consistency."""
    if pd.isna(x) or x is None:
        return None
    try:
        # Convert to string, remove commas, and strip whitespace
        cleaned_x = str(x).replace(',', '').strip()
        return float(cleaned_x)
    except (ValueError, TypeError):
        return None

def normalize_bank_key(raw_key):
    """Normalizes bank names to a consistent short code, using fuzzy matching."""
    raw_key_lower = str(raw_key).lower().strip()
    replacements = {
        'ncba bank kenya plc': 'ncba',
        'ncba bank': 'ncba',
        'equity bank': 'equity',
        'i&m bank': 'i&m',
        'central bank of kenya': 'cbk',
        'kenya commercial bank': 'kcb',
        'kcb bank': 'kcb',
        'sbm bank (kenya) limited': 'sbm',
        'sbm bank': 'sbm',
        'absa bank': 'absa',
        'kingdom bank': 'kingdom'
    }

    # First, try direct replacement
    for long, short in replacements.items():
        if raw_key_lower.startswith(long):
            return raw_key_lower.replace(long, short).strip()

    # If no direct match, try fuzzy matching against known short codes/replacements
    all_bank_names = list(replacements.values()) + list(replacements.keys())
    all_bank_names = list(set(all_bank_names)) # Ensure uniqueness

    match = process.extractOne(raw_key_lower, all_bank_names, scorer=fuzz.ratio)
    if match and match[1] >= FUZZY_MATCH_THRESHOLD:
        for long, short in replacements.items():
            if match[0].startswith(long):
                return short
        return match[0]
    return raw_key_lower # Return original if no good fuzzy match

def resolve_amount_column(columns, operation):
    """Identifies the amount column based on the operation (credit/debit)."""
    columns_lower = [col.lower() for col in columns]
    if operation.lower() == 'credit':
        candidates = ['credit', 'deposit']
    elif operation.lower() == 'debit':
        candidates = ['debit', 'withdrawal']
    else:
        candidates = ['amount', 'value', 'credit', 'deposit', 'debit', 'withdrawal'] # Fallback for general amount

    for key in candidates:
        if key in columns_lower:
            return columns[columns_lower.index(key)]
    return None

def resolve_date_column(columns):
    """Identifies the date column from a list of column names, prioritizing common formats."""
    for candidate in ['Value Date', 'Transaction Date', 'MyUnknownColumn', 'Transaction date', 'Date', 'Activity Date']:
        if candidate in columns:
            return candidate
    return None

def get_amount_columns(columns):
    """Returns a list of potential amount columns."""
    return [col for col in columns if col.lower() in ['deposit', 'credit', 'withdrawal', 'debit', 'amount', 'value']]

def get_description_columns(columns):
    """Identifies the description column from a list of column names."""
    for desc in ['Transaction details','Transaction', 'Customer reference','Narration',
                 'Transaction Details', 'Detail',  'Transaction Remarks:',
                 'TransactionDetails', 'Description', 'Narrative', 'Remarks']:
        if desc in columns:
            return desc
    return None

def parse_date(date_str_raw):
    """Parses a date string into a datetime object using predefined formats."""
    if not isinstance(date_str_raw, str):
        return None
    # Attempt to parse as date only, stripping time if present
    date_str = date_str_raw.split()[0].strip()
    for fmt in DATE_FORMATS: # Use the global DATE_FORMATS
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return None

In [40]:
# === Matching Logic ===
matched = []
unmatched = []

In [41]:
adjustment_rows = fx_trade_df.to_dict(orient='records')
adjustment_rows

[{'Request ID': 'd000105eaa68f918010003967',
  'Intermediary Account': 'Equity Bank',
  'Payment Channel': nan,
  'Counterparty Bank': 'Choice Microfinance Bank',
  'Counterparty Account ID': '1',
  'Counterparty Name': 'CELESTINE MUGAMBI KIBAARA',
  'Counterparty Sub-account': nan,
  'Transfer Reference No.': 'CELESTINE MUGAMBI KIBAARA  JULY 2025 CASH ADVANCE',
  'Transaction Narrative': 'CELESTINE MUGAMBI KIBAARA  JULY 2025 CASH ADVANCE  6,000',
  'TX ID': 'CTRANS0213eb866c802001',
  'Customer Account Number': 46010000015565,
  'Account Name': 'CELESTINE MUGAMBI KIBAARA',
  'Account Channel': 'Choice Personal',
  'Operation': 'debit',
  'Product': 'OP0024 - C - Account Adjustment - Credit Balance',
  'Currency': 'KES',
  'Amount': 4654080,
  'Status': 'Successful',
  'Requested At': '2025-07-11 17:49:20',
  'Completed At': '2025-07-11 17:49:53'},
 {'Request ID': 'd000105ea9f1e624010000c0b',
  'Intermediary Account': 'Unclaimed Funds',
  'Payment Channel': 'RTGS',
  'Counterparty Bank

In [42]:
from datetime import datetime
import pandas as pd
from fuzzywuzzy import fuzz

def process_foreign_currency_adjustment(
    adj_row: pd.Series,
    all_bank_dfs: dict,
    unmatched_list: list,
    matched_list: list,
    action_type: str,
    amount_field: str,
    bank_field: str,
    currency_field: str,
    date_field: str,
    date_tolerance_days: int = 3,
    amount_tolerance: float = 1.0,
    debug: bool = True
) -> bool:
    def log(msg):
        if debug:
            print(msg)

    log(f"\n🔍 Processing Adjustment row: {adj_row.to_dict()}")

    amount = safe_float(adj_row.get(amount_field))
    if amount is None:
        log("❌ Skipping row due to invalid amount")
        return False

    parsed_date = pd.to_datetime(adj_row.get(date_field), errors='coerce')
    if pd.isna(parsed_date):
        log("❌ Skipping row due to invalid or missing date")
        return False

    bank_name_raw = str(adj_row.get(bank_field, '')).strip()
    currency = str(adj_row.get(currency_field, '')).strip().upper()

    if not bank_name_raw or not currency:
        log("❌ Missing bank or currency")
        return False

    normalized_bank_key = normalize_bank_key(bank_name_raw)
    expected_bank_df_key = f"{normalized_bank_key} {currency}".lower()

    log(f"🔍 Matching against bank key: {expected_bank_df_key}")

    found_match = False
    target_bank_df_key = None

    for bank_df_key in all_bank_dfs.keys():
        bank_key_parts = bank_df_key.split('_')
        bank_key_base = bank_key_parts[0]
        bank_key_currency = bank_key_parts[1].upper() if len(bank_key_parts) > 1 else ""

        log(f"👉 Checking against: {bank_df_key}")

        if (expected_bank_df_key.startswith(bank_key_base) and currency == bank_key_currency):
            target_bank_df_key = bank_df_key
            log(f"✅ Exact prefix match found: {target_bank_df_key}")
            break

        score = fuzz.ratio(expected_bank_df_key, bank_df_key)
        log(f"🔁 Fuzzy score with '{bank_df_key}': {score}")
        if score >= FUZZY_MATCH_THRESHOLD:
            target_bank_df_key = bank_df_key
            log(f"✅ Fuzzy match found: {target_bank_df_key}")
            break

    if not target_bank_df_key:
        log("❌ No matching bank statement found")
        unmatched_list.append({
            'Date': parsed_date.strftime('%Y-%m-%d'),
            'Bank Table (Expected)': expected_bank_df_key,
            'Action Type': action_type,
            'Amount': amount,
            'Status': 'No Bank Statement Found',
            'Source Column': bank_field
        })
        return False

    bank_df = all_bank_dfs[target_bank_df_key]
    bank_df_columns = bank_df.columns.tolist()

    date_column = resolve_date_column(bank_df_columns)
    amount_column = resolve_amount_column(bank_df_columns, action_type)

    log(f"📅 Using date column: {date_column} | 💰 Using amount column: {amount_column}")

    if not date_column or not amount_column:
        log("❌ Missing date or amount column in bank data")
        unmatched_list.append({
            'Date': parsed_date.strftime('%Y-%m-%d'),
            'Bank Table (Expected)': target_bank_df_key,
            'Action Type': action_type,
            'Amount': amount,
            'Status': 'Missing Date/Amount Column in Bank Statement',
            'Source Column': bank_field
        })
        return False

    # Normalize parsed date and compare using pd.Timestamp
    ref_date = parsed_date.normalize()
    bank_df['_ParsedDate'] = bank_df[date_column].apply(parse_date)

    valid_dates_df = bank_df[bank_df['_ParsedDate'].notna()]
    date_matches = valid_dates_df[
        valid_dates_df['_ParsedDate'].between(
            ref_date - pd.Timedelta(days=date_tolerance_days),
            ref_date + pd.Timedelta(days=date_tolerance_days)
        )
    ]

    log(f"🔎 Found {len(date_matches)} date matches in bank statement")

    for idx, bank_row in date_matches.iterrows():
        bank_amt_raw = bank_row[amount_column] if amount_column in bank_row else None
        bank_amt = safe_float(bank_amt_raw)
        log(f"\nBank amount raw: {bank_amt_raw}, converted: {bank_amt}")

        if bank_amt is not None and abs(bank_amt) > 1.0:
            log("✅ Match found!")
            matched_list.append({
                'Date': parsed_date.strftime('%Y-%m-%d'),
                'Bank Table': target_bank_df_key,
                'Action Type': action_type,
                'Adjustment Amount': amount,
                'Bank Statement Amount': bank_amt,
                'Matched In Column': amount_column,
                'Date Column Used': date_column,
                'Source Column': bank_field
            })
            found_match = True
            break

    if not found_match:
        log("❌ No match found in statement rows")
        unmatched_list.append({
            'Date': parsed_date.strftime('%Y-%m-%d'),
            'Bank Table (Expected)': target_bank_df_key,
            'Action Type': action_type,
            'Amount': amount,
            'Status': 'Not Found in Bank Statement (Amount or No Match)',
            'Source Column': bank_field
        })

    return found_match


In [22]:
buy_match_count = 0
sell_match_count = 0
unmatched_buy = []
unmatched_sell = []
unmatched_bank_records = []
matched_buy = []
matched_sell = []
matched_set = set()  # To track matched bank records to avoid double counting

if not fx_trade_df.empty and bank_dfs:
    print("\n--- Starting Reconciliation Process ---")

    fx_trade_df.columns = fx_trade_df.columns.str.strip()

    for index, row in fx_trade_df.iterrows():
        action_type = str(row.get('Operation', '')).strip()
        status = str(row.get('Status', '')).strip().lower()
        if status == 'cancelled':
            continue

        amount = safe_float(row.get('Amount'))
        bank_info = row.get('Intermediary Account')
        currency = str(row.get('Currency', '')).strip().upper()
        p_createdAt = parse_date(row.get('Completed At'))

        # --- BUY SIDE ---
        if process_foreign_currency_adjustment(
            adj_row=row,
            all_bank_dfs=bank_dfs,
            unmatched_list=unmatched_buy,
            matched_list=matched_buy,
            action_type="Debit",
            amount_field="Amount",
            bank_field="Intermediary Account",
            currency_field="Currency",
            date_field="Completed At",
            date_tolerance_days=1,
            amount_tolerance=1.0,
            debug=True
        ):
            buy_match_count += 1
            print(bank_info, " : ", amount, " : ", p_createdAt)
            if bank_info and amount is not None and isinstance(p_createdAt, datetime):
                parts = bank_info.lower().split(' ')
                if len(parts) >= 2:
                    key_bank = normalize_bank_key(parts[0].strip())
                    key_currency = currency
                    match_key = (
                        f"{key_bank.replace('_bank', '')} {key_currency}",
                        round(amount, 2),
                        p_createdAt.strftime('%Y-%d-%m')
                    )
                    matched_set.add(match_key)

        # --- SELL SIDE ---
        if process_foreign_currency_adjustment(
            adj_row=row,
            all_bank_dfs=bank_dfs,
            unmatched_list=unmatched_sell,
            matched_list=matched_sell,
            action_type="Credit",
            amount_field="Amount",
            bank_field="Intermediary Account",
            currency_field="Currency",
            date_field="Completed At",
            date_tolerance_days=1,
            amount_tolerance=1.0,
            debug=True
        ):
            sell_match_count += 1
            print(bank_info, " : ", amount, " : ", p_createdAt)


            if bank_info and amount is not None and isinstance(p_createdAt, datetime):
                parts = bank_info.lower().split(' ')

                if len(parts) >= 2:
                    key_bank = normalize_bank_key(parts[0].strip())
                    key_currency = currency
                    matched_entry = next(
                        (item for item in matched_sell if item['Date'] == p_createdAt.strftime('%Y-%m-%d') and round(item['Adjustment Amount'], 2) == round(amount, 2)),
                        None
                    )
                    if matched_entry:
                        match_key = (
                            f"{key_bank.replace('_bank', '')} {key_currency}",
                            round(matched_entry['Bank Statement Amount'], 2),
                            p_createdAt.strftime('%Y-%d-%m')
                        )
                        matched_set.add(match_key)

    # --- Scan Bank Tables for Unmatched Entries ---
    for bank_key, bank_df in bank_dfs.items():
        print(f"🔎 Scanning bank table: {bank_key}")
        bank_df.columns = bank_df.columns.str.strip()
        date_col = resolve_date_column(bank_df.columns.tolist())
        amount_cols = get_amount_columns(bank_df.columns.tolist())
        description_col = get_description_columns(bank_df.columns.tolist())

        if not date_col or not amount_cols or not description_col:
            print(f"⚠️ Skipping {bank_key}: Missing required columns.")
            continue

        bank_df['_ParsedDate'] = bank_df[date_col].apply(parse_date)

        parts = bank_key.lower().split(' ')
        key_bank = normalize_bank_key(parts[0].strip())
        key_currency = currency

        for idx, row in bank_df.iterrows():
            row_date = row.get('_ParsedDate')
            if not isinstance(row_date, datetime):
                continue

            description = str(row.get(description_col, '')).strip()

            for amt_col in amount_cols:
                amt_val = safe_float(row.get(amt_col))
                if amt_val is None or abs(amt_val) < 0.01:
                    continue

                rounded_amt = round(amt_val, 2)
                match_key = (
                    f"{key_bank} {key_currency}",
                    rounded_amt,
                    row_date.strftime('%Y-%d-%m')
                )

                print("\n🔍 Checking match_key:", match_key, " :set : ", matched_set )
                if match_key in matched_set:
                    print("✅ Found match.")
                else:
                    print("❌ No match found.")
                    print(f"🔍 Similar matches: {[x for x in matched_set if x[0] == match_key[0] and x[1] == match_key[1]]}")

                    unmatched_bank_records.append({
                        'Bank Table': bank_key,
                        'Date': row_date.strftime('%Y-%m-%d'),
                        'Description': description,
                        'Transaction Type (Column)': amt_col,
                        'Amount': rounded_amt
                    })

# Create summary DataFrames
unmatched_buy_df = pd.DataFrame(unmatched_buy)
unmatched_sell_df = pd.DataFrame(unmatched_sell)
matched_buy_df = pd.DataFrame(matched_buy)
matched_sell_df = pd.DataFrame(matched_sell)
unmatched_bank_df = pd.DataFrame(unmatched_bank_records)


In [43]:
from datetime import datetime, timedelta


In [ ]:


matched_adjustments = []
unmatched_adjustments = []
unmatched_bank_records = []
matched_bank_keys = set() # To track matched bank records by a unique key (bank_key, date, amount)

def reconcile_adjustment_row(
    adj_row: pd.Series,
    all_bank_dfs: dict,
    mode: str, # 'local' or 'foreign'
    date_tolerance_days: int = 3,
    amount_tolerance: float = 1.0, # Absolute tolerance for amount matching
    debug: bool = True # Control verbose printing
) -> bool:
    """
    Attempts to reconcile a single adjustment row against all uploaded bank statements.
    Returns True if a match is found, False otherwise.
    Appends to global matched_adjustments or unmatched_adjustments lists.
    """
    global matched_bank_keys # Declare global to modify the set

    def log(msg):
        if debug:
            print(msg)

    log(f"\n🔍 Processing Adjustment row: {adj_row.to_dict()}")

    amount = safe_float(adj_row.get('Amount'))
    if amount is None or pd.isna(amount) or abs(amount) < 0.01: # Add check for very small amounts
        log("❌ Skipping row due to invalid or insignificant amount")
        return False

    parsed_date = parse_date(adj_row.get('Completed At'))
    if pd.isna(parsed_date) or parsed_date is None:
        log("❌ Skipping row due to invalid or missing date ('Completed At')")
        return False

    # FIX: Replace .normalize() for datetime.datetime object
    # The .normalize() method is for pandas.Timestamp. For datetime.datetime,
    # to get the date part only (time set to midnight), we can construct a new datetime object.
    ref_date = datetime(parsed_date.year, parsed_date.month, parsed_date.day)


    operation = str(adj_row.get('Operation', '')).strip().lower()
    if operation not in ['credit', 'debit']:
        log(f"❌ Skipping row due to unrecognised operation: '{operation}'")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': f'Unrecognised operation: {operation}'})
        return False

    status = str(adj_row.get('Status', '')).strip().lower()
    if (mode == 'local' and status != 'successful') or \
       (mode == 'foreign' and status != 'completed'):
        log(f"❌ Skipping row due for mode '{mode}' and status '{status}'")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': f'Skipped due to status "{status}" for mode "{mode}"'})
        return False

    intermediary_account = str(adj_row.get('Intermediary Account', '')).strip()
    currency = str(adj_row.get('Currency', '')).strip().upper()

    expected_bank_df_key_base = None

    if mode == 'local':
        # For local, bank_name from 'Intermediary Account' and currency
        bank_name_for_key = normalize_bank_key(intermediary_account)
        if not bank_name_for_key:
            log(f"❌ Could not normalize bank name for local mode: '{intermediary_account}'")
            unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'Could not normalize bank name for local mode'})
            return False
        expected_bank_df_key_base = f"{bank_name_for_key.lower()} {currency.lower()}"
    elif mode == 'foreign':
        # For foreign, parse 'Intermediary Account' as 'BankName - Currency'
        parts = intermediary_account.split('-')
        if len(parts) < 2:
            log(f"❌ Skipping row due to malformed foreign intermediary account: '{intermediary_account}'")
            unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'Malformed foreign intermediary account'})
            return False
        bank_name_raw = parts[0].strip()
        currency_raw = parts[1].strip().upper()

        bank_name_for_key = normalize_bank_key(bank_name_raw)
        if not bank_name_for_key:
            log(f"❌ Could not normalize bank name for foreign mode: '{bank_name_raw}'")
            unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'Could not normalize bank name for foreign mode'})
            return False

        if currency != currency_raw:
             log(f"⚠️ Currency mismatch: Adjustment currency '{currency}' vs Intermediary Account currency '{currency_raw}'")
             # Proceeding with Intermediary Account currency as it's more specific for the bank key
        expected_bank_df_key_base = f"{bank_name_for_key.lower()} {currency_raw.lower()}"
    else:
        log(f"❌ Invalid reconciliation mode: {mode}")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': f'Invalid mode: {mode}'})
        return False

    log(f"🔍 Expected Bank DF Key Base: {expected_bank_df_key_base}")

    target_bank_df_key = None
    for bank_df_key in all_bank_dfs.keys():
        bank_df_key_normalized = bank_df_key.lower().replace('_', ' ') # Normalize keys for comparison
        log(f"Comparing '{expected_bank_df_key_base}' with '{bank_df_key_normalized}'")
        if bank_df_key_normalized.startswith(expected_bank_df_key_base):
            target_bank_df_key = bank_df_key
            log(f"✅ Exact prefix match found for bank DF key: {target_bank_df_key}")
            break
        score = fuzz.ratio(expected_bank_df_key_base, bank_df_key_normalized)
        log(f"🔁 Fuzzy score with '{bank_df_key_normalized}': {score}")
        if score >= FUZZY_MATCH_THRESHOLD:
            target_bank_df_key = bank_df_key
            log(f"✅ Fuzzy match found for bank DF key: {target_bank_df_key}")
            break

    if not target_bank_df_key:
        log("❌ No matching bank statement found for this adjustment")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'No matching bank statement found'})
        return False

    bank_df = all_bank_dfs[target_bank_df_key]
    if bank_df.empty:
        log(f"⚠️ Bank statement '{target_bank_df_key}' is empty.")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': f'Target bank statement ({target_bank_df_key}) is empty'})
        return False

    bank_df_columns = bank_df.columns.tolist()

    date_column = resolve_date_column(bank_df_columns)
    amount_column = resolve_amount_column(bank_df_columns, operation) # Pass operation to resolve correct amount column

    log(f"📅 Using date column: {date_column} | 💰 Using amount column: {amount_column}")

    if not date_column or not amount_column:
        log("❌ Missing date or amount column in bank data for reconciliation")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'Missing date/amount column in bank statement'})
        return False

    # Convert bank statement date column to datetime objects
    bank_df['_ParsedDate'] = bank_df[date_column].apply(parse_date)

    # Filter by date tolerance
    date_matches_df = bank_df[
        (bank_df['_ParsedDate'].notna()) &
        (bank_df['_ParsedDate'].between(
            ref_date - timedelta(days=date_tolerance_days), # Use timedelta
            ref_date + timedelta(days=date_tolerance_days)  # Use timedelta
        ))
    ].copy() # Use .copy() to avoid SettingWithCopyWarning

    log(f"🔎 Found {len(date_matches_df)} date matches in bank statement '{target_bank_df_key}'")

    match_found = False
    for idx, bank_row in date_matches_df.iterrows():
        bank_amt_raw = bank_row.get(amount_column)
        bank_amt = safe_float(bank_amt_raw)

        if bank_amt is None:
            continue

        log(f"  Comparing bank amount {bank_amt} (from column '{amount_column}') with adjustment amount {amount}")

        # Check for absolute difference within tolerance
        if abs(bank_amt - amount) <= amount_tolerance:
            # Generate a unique key for the bank record to mark it as matched
            bank_record_key = (
                target_bank_df_key,
                bank_row['_ParsedDate'].strftime('%Y-%m-%d'),
                round(bank_amt, 2),
                operation # Include operation in key to differentiate debit/credit if amounts are similar
            )
            if bank_record_key not in matched_bank_keys:
                matched_adjustments.append({
                    'Adjustment_Date': parsed_date.strftime('%Y-%m-%d'),
                    'Adjustment_Amount': amount,
                    'Adjustment_Operation': operation,
                    'Adjustment_Intermediary_Account': intermediary_account,
                    'Adjustment_Currency': currency,
                    'Bank_Table': target_bank_df_key,
                    'Bank_Statement_Date': bank_row['_ParsedDate'].strftime('%Y-%m-%d'),
                    'Bank_Statement_Amount': bank_amt,
                    'Bank_Matched_Column': amount_column,
                    'Bank_Row_Index': idx # Keep original index for debugging/tracking
                })
                matched_bank_keys.add(bank_record_key)
                log("✅ Match found and recorded!")
                match_found = True
                break
            else:
                log("⚠️ Potential duplicate match skipped (bank record already matched).")
                continue # Keep looking for another match if this one was already used

    if not match_found:
        log("❌ No amount match found within tolerance for this adjustment.")
        unmatched_adjustments.append({**adj_row.to_dict(), 'Reason': 'No amount match in bank statement'})
    return match_found


# Main reconciliation execution block
# This replaces the logic in cells 51dc9d47 and 76217333

def perform_reconciliation():
    global matched_adjustments, unmatched_adjustments, unmatched_bank_records, matched_bank_keys
    matched_adjustments = []
    unmatched_adjustments = []
    unmatched_bank_records = []
    matched_bank_keys = set() # Reset for each reconciliation run

    # Assume fx_trade_df and bank_dfs are populated from previous steps.
    # If not, provide mock data or ensure the UI elements trigger their population.
    global fx_trade_df, bank_dfs # To ensure we are using the global variables populated by other UI interactions
    # Example mock data if not populated from UI for testing:
    # fx_trade_df = pd.DataFrame([
    #     {'Amount': 100.0, 'Completed At': '2023-01-10', 'Operation': 'Debit', 'Status': 'Successful', 'Intermediary Account': 'NCBA - KES', 'Currency': 'KES'},
    #     {'Amount': 50.0, 'Completed At': '2023-01-12', 'Operation': 'Credit', 'Status': 'Completed', 'Intermediary Account': 'Equity Bank - USD', 'Currency': 'USD'}
    # ])
    # bank_dfs = {
    #     'ncba kes': pd.DataFrame([
    #         {'Date': '2023-01-10', 'Debit': 100.0, 'Description': 'Payment'},
    #         {'Date': '2023-01-11', 'Debit': 20.0, 'Description': 'Fee'}
    #     ]),
    #     'equity usd': pd.DataFrame([
    #         {'Date': '2023-01-12', 'Credit': 50.0, 'Description': 'Deposit'},
    #         {'Date': '2023-01-13', 'Credit': 70.0, 'Description': 'Interest'}
    #     ])
    # }

    # For testing, ensure mode_selector is defined or hardcode it
    class MockModeSelector:
        def __init__(self, value):
            self.value = value
    # mode_selector = MockModeSelector('local') # Or 'foreign' for testing

    if fx_trade_df.empty:
        with fx_output:
            print("⚠️ FX Data is empty. Please upload and process FX Tracker data.")
        return
    if not bank_dfs:
        with bank_output:
            print("⚠️ No Bank Statements processed. Please upload and process bank data.")
        return

    # Check if mode_selector is defined, otherwise default or raise an error
    if 'mode_selector' not in globals() or not isinstance(mode_selector, widgets.Dropdown):
        # Fallback for when run in isolation, but in a full notebook, it should be a widget
        print("Error: 'mode_selector' widget not found. Please ensure all UI elements are defined.")
        return


    print("\n--- Starting Reconciliation Process ---")

    current_mode = mode_selector.value
    print(f"Reconciliation Mode: {current_mode.upper()}")

    # Process all adjustments first
    for index, row in fx_trade_df.iterrows():
        reconcile_adjustment_row(
            adj_row=row,
            all_bank_dfs=bank_dfs,
            mode=current_mode,
            date_tolerance_days=3, # Can be made a UI configurable parameter
            amount_tolerance=1.0, # Can be made a UI configurable parameter
            debug=True # Set to True for detailed debug output during reconciliation
        )

    # After processing all adjustments, identify unmatched bank records
    print("\n--- Identifying Unmatched Bank Records ---")
    for bank_key, bank_df in bank_dfs.items():
        if bank_df.empty:
            print(f"⚠️ Skipping empty bank statement: {bank_key}")
            continue

        bank_df_copy = bank_df.copy() # Work on a copy to avoid SettingWithCopyWarning
        bank_df_copy.columns = bank_df_copy.columns.str.strip()
        date_col = resolve_date_column(bank_df_copy.columns.tolist())
        amount_cols = get_amount_columns(bank_df_copy.columns.tolist())
        description_col = get_description_columns(bank_df_copy.columns.tolist())

        if not date_col or not amount_cols or not description_col:
            print(f"⚠️ Skipping '{bank_key}': Missing required columns (Date, Amount, or Description).")
            continue

        bank_df_copy['_ParsedDate'] = bank_df_copy[date_col].apply(parse_date)

        for idx, row in bank_df_copy.iterrows():
            row_date = row.get('_ParsedDate')
            if not isinstance(row_date, datetime) or pd.isna(row_date):
                continue

            description = str(row.get(description_col, '')).strip()

            is_matched_in_any_way = False
            found_amt_col = None
            found_amt_val = None

            for amt_col in amount_cols:
                amt_val = safe_float(row.get(amt_col))
                if amt_val is None or abs(amt_val) < 0.01:
                    continue

                rounded_amt = round(amt_val, 2)
                # Need to determine the 'operation' (debit/credit) based on the column or sign
                operation_for_key = 'debit' if 'debit' in amt_col.lower() or amt_val < 0 else 'credit'
                if 'credit' in amt_col.lower():
                    operation_for_key = 'credit'
                elif 'debit' in amt_col.lower():
                    operation_for_key = 'debit'

                bank_record_key = (
                    bank_key,
                    row_date.strftime('%Y-%m-%d'),
                    rounded_amt,
                    operation_for_key
                )
                if bank_record_key in matched_bank_keys:
                    is_matched_in_any_way = True
                    found_amt_col = amt_col
                    found_amt_val = rounded_amt
                    break # Break from amount_cols loop if already matched
            if is_matched_in_any_way:
                continue # This bank record was matched, move to the next bank record

            # If we reach here, the bank record was not matched by any adjustment
            # We still need to pick an amount column if there are multiple.
            # For unmatched, we can just pick the first valid amount column found.
            # It's also possible a record has multiple amount columns (e.g., credit and debit are distinct columns)
            # and only one is relevant for its actual transaction.
            # A more sophisticated approach might aggregate or sum if they represent a net change.
            final_amt_col_for_unmatched = None
            final_amt_val_for_unmatched = None
            for amt_col in amount_cols:
                amt_val = safe_float(row.get(amt_col))
                if amt_val is not None and abs(amt_val) >= 0.01:
                    final_amt_col_for_unmatched = amt_col
                    final_amt_val_for_unmatched = round(amt_val, 2)
                    break # Take the first non-zero valid amount

            if final_amt_val_for_unmatched is not None:
                unmatched_bank_records.append({
                    'Bank_Table': bank_key,
                    'Date': row_date.strftime('%Y-%m-%d'),
                    'Description': description,
                    'Transaction_Type_Column': final_amt_col_for_unmatched,
                    'Amount': final_amt_val_for_unmatched,
                    'Original_Row_Index': idx
                })

    # Create summary DataFrames
    df_matched_adjustments = pd.DataFrame(matched_adjustments)
    df_unmatched_adjustments = pd.DataFrame(unmatched_adjustments)
    df_unmatched_bank_records = pd.DataFrame(unmatched_bank_records)

    print("\n===== RECONCILIATION SUMMARY =====")
    print(f"✅ Total Adjustments Matched: {len(df_matched_adjustments)}")
    print(f"❌ Total Adjustments Unmatched: {len(df_unmatched_adjustments)}")
    print(f"📄 Total Unmatched Bank Records: {len(df_unmatched_bank_records)}")

    with Output(): # Use an Output widget for displaying results to avoid cluttering the main cell output
        print("\n--- Matched Adjustments ---")
        if not df_matched_adjustments.empty:
            display(df_matched_adjustments.head())
            display(generate_download_link(df_matched_adjustments, "matched_adjustments.csv"))
        else:
            print("No matched adjustments.")

        print("\n--- Unmatched Adjustments ---")
        if not df_unmatched_adjustments.empty:
            display(df_unmatched_adjustments.head())
            display(generate_download_link(df_unmatched_adjustments, "unmatched_adjustments.csv"))
        else:
            print("No unmatched adjustments.")

        print("\n--- Unmatched Bank Records ---")
        if not df_unmatched_bank_records.empty:
            display(df_unmatched_bank_records.head())
            display(generate_download_link(df_unmatched_bank_records, "unmatched_bank_records.csv"))
        else:
            print("No unmatched bank records.")

# Add a button to trigger the reconciliation
reconcile_btn = widgets.Button(description='Perform Reconciliation', button_style='info')
reconciliation_output = Output() # Output widget specifically for reconciliation results

def on_reconcile_click(b):
    with reconciliation_output:
        clear_output()
        perform_reconciliation()

reconcile_btn.on_click(on_reconcile_click)

# Display the button and output area
print("\n--- Trigger Reconciliation ---")
# Assume mode_selector is defined globally by other UI code
mode_selector = widgets.Dropdown(
    options=['local', 'foreign'],
    value='local', # Default value
    description='Mode:',
    disabled=False,
)
display(mode_selector, reconcile_btn, reconciliation_output)


--- Trigger Reconciliation ---


Dropdown(description='Mode:', options=('local', 'foreign'), value='local')

Button(button_style='info', description='Perform Reconciliation', style=ButtonStyle())

Output()

In [ ]:


# Global DataFrames to store reconciliation results for analysis
df_matched_adjustments = pd.DataFrame()
df_unmatched_adjustments = pd.DataFrame()
df_unmatched_bank_records = pd.DataFrame()

matched_adjustments_list = []
unmatched_adjustments_list = []
unmatched_bank_records_list = []
matched_bank_keys = set()


def reconcile_adjustment_row(
    adj_row: pd.Series,
    all_bank_dfs: dict,
    mode: str, # 'local' or 'foreign'
    date_tolerance_days: int = 3,
    amount_tolerance: float = 1.0, # Absolute tolerance for amount matching
    debug: bool = False # Control verbose printing
) -> bool:
    """
    Attempts to reconcile a single adjustment row against all uploaded bank statements.
    Returns True if a match is found, False otherwise.
    Appends to global matched_adjustments_list or unmatched_adjustments_list lists.
    """
    global matched_bank_keys, matched_adjustments_list, unmatched_adjustments_list

    def log(msg):
        if debug:
            print(msg)

    log(f"\n🔍 Processing Adjustment row: {adj_row.to_dict()}")

    amount = safe_float(adj_row.get('Amount'))
    if amount is None or pd.isna(amount) or abs(amount) < 0.01:
        log("❌ Skipping row due to invalid or insignificant amount")
        return False

    parsed_date = parse_date(adj_row.get('Completed At'))
    if pd.isna(parsed_date) or parsed_date is None:
        log("❌ Skipping row due to invalid or missing date ('Completed At')")
        return False

    ref_date = datetime(parsed_date.year, parsed_date.month, parsed_date.day)

    operation = str(adj_row.get('Operation', '')).strip().lower()
    if operation not in ['credit', 'debit']:
        log(f"❌ Skipping row due to unrecognised operation: '{operation}'")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': f'Unrecognised operation: {operation}'})
        return False

    status = str(adj_row.get('Status', '')).strip().lower()
    if (mode == 'local' and status != 'successful') or \
       (mode == 'foreign' and status != 'completed'):
        log(f"❌ Skipping row due for mode '{mode}' and status '{status}'")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': f'Skipped due to status "{status}" for mode "{mode}"'})
        return False

    intermediary_account = str(adj_row.get('Intermediary Account', '')).strip()
    currency = str(adj_row.get('Currency', '')).strip().upper()

    expected_bank_df_key_base = None

    if mode == 'local':
        bank_name_for_key = normalize_bank_key(intermediary_account)
        if not bank_name_for_key:
            log(f"❌ Could not normalize bank name for local mode: '{intermediary_account}'")
            unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'Could not normalize bank name for local mode'})
            return False
        expected_bank_df_key_base = f"{bank_name_for_key.lower()} {currency.lower()}"
    elif mode == 'foreign':
        parts = intermediary_account.split('-')
        if len(parts) < 2:
            log(f"❌ Skipping row due to malformed foreign intermediary account: '{intermediary_account}'")
            unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'Malformed foreign intermediary account'})
            return False
        bank_name_raw = parts[0].strip()
        currency_raw = parts[1].strip().upper()

        bank_name_for_key = normalize_bank_key(bank_name_raw)
        if not bank_name_for_key:
            log(f"❌ Could not normalize bank name for foreign mode: '{bank_name_raw}'")
            unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'Could not normalize bank name for foreign mode'})
            return False

        if currency != currency_raw:
             log(f"⚠️ Currency mismatch: Adjustment currency '{currency}' vs Intermediary Account currency '{currency_raw}'")
        expected_bank_df_key_base = f"{bank_name_for_key.lower()} {currency_raw.lower()}"
    else:
        log(f"❌ Invalid reconciliation mode: {mode}")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': f'Invalid mode: {mode}'})
        return False

    log(f"🔍 Expected Bank DF Key Base: {expected_bank_df_key_base}")

    target_bank_df_key = None
    for bank_df_key in all_bank_dfs.keys():
        bank_df_key_normalized = bank_df_key.lower().replace('_', ' ')
        log(f"Comparing '{expected_bank_df_key_base}' with '{bank_df_key_normalized}'")
        if bank_df_key_normalized.startswith(expected_bank_df_key_base):
            target_bank_df_key = bank_df_key
            log(f"✅ Exact prefix match found for bank DF key: {target_bank_df_key}")
            break
        score = fuzz.ratio(expected_bank_df_key_base, bank_df_key_normalized)
        log(f"🔁 Fuzzy score with '{bank_df_key_normalized}': {score}")
        if score >= FUZZY_MATCH_THRESHOLD:
            target_bank_df_key = bank_df_key
            log(f"✅ Fuzzy match found for bank DF key: {target_bank_df_key}")
            break

    if not target_bank_df_key:
        log("❌ No matching bank statement found for this adjustment")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'No matching bank statement found'})
        return False

    bank_df = all_bank_dfs[target_bank_df_key]
    if bank_df.empty:
        log(f"⚠️ Bank statement '{target_bank_df_key}' is empty.")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': f'Target bank statement ({target_bank_df_key}) is empty'})
        return False

    bank_df_columns = bank_df.columns.tolist()

    date_column = resolve_date_column(bank_df_columns)
    amount_column = resolve_amount_column(bank_df_columns, operation)

    log(f"📅 Using date column: {date_column} | 💰 Using amount column: {amount_column}")

    if not date_column or not amount_column:
        log("❌ Missing date or amount column in bank data for reconciliation")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'Missing date/amount column in bank statement'})
        return False

    bank_df['_ParsedDate'] = bank_df[date_column].apply(parse_date)

    date_matches_df = bank_df[
        (bank_df['_ParsedDate'].notna()) &
        (bank_df['_ParsedDate'].between(
            ref_date - timedelta(days=date_tolerance_days),
            ref_date + timedelta(days=date_tolerance_days)
        ))
    ].copy()

    log(f"🔎 Found {len(date_matches_df)} date matches in bank statement '{target_bank_df_key}'")

    match_found = False
    for idx, bank_row in date_matches_df.iterrows():
        bank_amt_raw = bank_row.get(amount_column)
        bank_amt = safe_float(bank_amt_raw)

        if bank_amt is None:
            continue

        log(f"  Comparing bank amount {bank_amt} (from column '{amount_column}') with adjustment amount {amount}")

        if abs(bank_amt - amount) <= amount_tolerance:
            bank_record_key = (
                target_bank_df_key,
                bank_row['_ParsedDate'].strftime('%Y-%m-%d'),
                round(bank_amt, 2),
                operation
            )
            if bank_record_key not in matched_bank_keys:
                matched_adjustments_list.append({
                    'Adjustment_Date': parsed_date.strftime('%Y-%m-%d'),
                    'Adjustment_Amount': amount,
                    'Adjustment_Operation': operation,
                    'Adjustment_Intermediary_Account': intermediary_account,
                    'Adjustment_Currency': currency,
                    'Bank_Table': target_bank_df_key,
                    'Bank_Statement_Date': bank_row['_ParsedDate'].strftime('%Y-%m-%d'),
                    'Bank_Statement_Amount': bank_amt,
                    'Bank_Matched_Column': amount_column,
                    'Bank_Row_Index': idx
                })
                matched_bank_keys.add(bank_record_key)
                log("✅ Match found and recorded!")
                match_found = True
                break
            else:
                log("⚠️ Potential duplicate match skipped (bank record already matched).")
                continue

    if not match_found:
        log("❌ No amount match found within tolerance for this adjustment.")
        unmatched_adjustments_list.append({**adj_row.to_dict(), 'Reason': 'No amount match in bank statement'})
    return match_found


# Main reconciliation execution block
def perform_reconciliation():
    global df_matched_adjustments, df_unmatched_adjustments, df_unmatched_bank_records
    global matched_adjustments_list, unmatched_adjustments_list, unmatched_bank_records_list, matched_bank_keys

    # Reset lists for new reconciliation run
    matched_adjustments_list = []
    unmatched_adjustments_list = []
    unmatched_bank_records_list = []
    matched_bank_keys = set()

    # Ensure fx_trade_df and bank_dfs are populated. If not, this function needs to be called
    # after the file upload/processing steps in the actual notebook.
    global fx_trade_df, bank_dfs

    if fx_trade_df.empty:
        with fx_output:
            print("⚠️ FX Data is empty. Please upload and process FX Tracker data.")
        return
    if not bank_dfs:
        with bank_output:
            print("⚠️ No Bank Statements processed. Please upload and process bank data.")
        return

    # Assuming mode_selector is a globally defined ipywidget.Dropdown
    if 'mode_selector' not in globals() or not isinstance(mode_selector, widgets.Dropdown):
        print("Error: 'mode_selector' widget not found. Ensure all UI elements are defined.")
        return

    print("\n--- Starting Reconciliation Process ---")
    current_mode = mode_selector.value
    print(f"Reconciliation Mode: {current_mode.upper()}")

    for index, row in fx_trade_df.iterrows():
        reconcile_adjustment_row(
            adj_row=row,
            all_bank_dfs=bank_dfs,
            mode=current_mode,
            date_tolerance_days=3,
            amount_tolerance=1.0,
            debug=True
        )

    print("\n--- Identifying Unmatched Bank Records ---")
    for bank_key, bank_df in bank_dfs.items():
        if bank_df.empty:
            print(f"⚠️ Skipping empty bank statement: {bank_key}")
            continue

        bank_df_copy = bank_df.copy()
        bank_df_copy.columns = bank_df_copy.columns.str.strip()
        date_col = resolve_date_column(bank_df_copy.columns.tolist())
        amount_cols = get_amount_columns(bank_df_copy.columns.tolist())
        description_col = get_description_columns(bank_df_copy.columns.tolist())

        if not date_col or not amount_cols or not description_col:
            print(f"⚠️ Skipping '{bank_key}': Missing required columns (Date, Amount, or Description).")
            continue

        bank_df_copy['_ParsedDate'] = bank_df_copy[date_col].apply(parse_date)

        for idx, row in bank_df_copy.iterrows():
            row_date = row.get('_ParsedDate')
            if not isinstance(row_date, datetime) or pd.isna(row_date):
                continue

            description = str(row.get(description_col, '')).strip()

            is_matched_in_any_way = False
            for amt_col in amount_cols:
                amt_val = safe_float(row.get(amt_col))
                if amt_val is None or abs(amt_val) < 0.01:
                    continue

                rounded_amt = round(amt_val, 2)
                operation_for_key = 'debit' if 'debit' in amt_col.lower() or amt_val < 0 else 'credit'
                if 'credit' in amt_col.lower():
                    operation_for_key = 'credit'
                elif 'debit' in amt_col.lower():
                    operation_for_key = 'debit'

                bank_record_key = (
                    bank_key,
                    row_date.strftime('%Y-%m-%d'),
                    rounded_amt,
                    operation_for_key
                )
                if bank_record_key in matched_bank_keys:
                    is_matched_in_any_way = True
                    break
            if is_matched_in_any_way:
                continue

            final_amt_col_for_unmatched = None
            final_amt_val_for_unmatched = None
            for amt_col in amount_cols:
                amt_val = safe_float(row.get(amt_col))
                if amt_val is not None and abs(amt_val) >= 0.01:
                    final_amt_col_for_unmatched = amt_col
                    final_amt_val_for_unmatched = round(amt_val, 2)
                    break

            if final_amt_val_for_unmatched is not None:
                unmatched_bank_records_list.append({
                    'Bank_Table': bank_key,
                    'Date': row_date.strftime('%Y-%m-%d'),
                    'Description': description,
                    'Transaction_Type_Column': final_amt_col_for_unmatched,
                    'Amount': final_amt_val_for_unmatched,
                    'Original_Row_Index': idx
                })

    # Convert lists to DataFrames and assign to global variables
    df_matched_adjustments = pd.DataFrame(matched_adjustments_list)
    df_unmatched_adjustments = pd.DataFrame(unmatched_adjustments_list)
    df_unmatched_bank_records = pd.DataFrame(unmatched_bank_records_list)

    print("\n===== RECONCILIATION SUMMARY =====")
    print(f"✅ Total Adjustments Matched: {len(df_matched_adjustments)}")
    print(f"❌ Total Adjustments Unmatched: {len(df_unmatched_adjustments)}")
    print(f"📄 Total Unmatched Bank Records: {len(df_unmatched_bank_records)}")

    
    print("\n--- Matched Adjustments ---")
    if not df_matched_adjustments.empty:
        display(df_matched_adjustments.head())
        display(generate_download_link(df_matched_adjustments, "matched_adjustments.csv"))
    else:
        print("No matched adjustments.")

    print("\n--- Unmatched Adjustments ---")
    if not df_unmatched_adjustments.empty:
        display(df_unmatched_adjustments.head())
        display(generate_download_link(df_unmatched_adjustments, "unmatched_adjustments.csv"))
    else:
        print("No unmatched adjustments.")

    print("\n--- Unmatched Bank Records ---")
    if not df_unmatched_bank_records.empty:
        display(df_unmatched_bank_records.head())
        display(generate_download_link(df_unmatched_bank_records, "unmatched_bank_records.csv"))
    else:
        print("No unmatched bank records.")

# Add a button to trigger the reconciliation
reconcile_btn = widgets.Button(description='Perform Reconciliation', button_style='info')
reconciliation_output = Output()

def on_reconcile_click(b):
    with reconciliation_output:
        clear_output()
        perform_reconciliation()

reconcile_btn.on_click(on_reconcile_click)

# Display the button and output area
print("\n--- Trigger Reconciliation ---")
mode_selector = widgets.Dropdown(
    options=['local', 'foreign'],
    value='local',
    description='Mode:',
    disabled=False,
)
display(mode_selector, reconcile_btn, reconciliation_output)



--- Trigger Reconciliation ---


Dropdown(description='Mode:', options=('local', 'foreign'), value='local')

Button(button_style='info', description='Perform Reconciliation', style=ButtonStyle())

Output()

In [ ]:

print("\n## 7. Data Science Analysis and Visualizations")
print("This section provides a deeper dive into the reconciliation results using `pandas` for analysis and `seaborn` for visually appealing graphs.")

def perform_data_analysis_and_visualizations():
    clear_output()
    print("--- Performing Data Analysis and Visualizations ---")

    # Ensure DataFrames are not empty before proceeding
    if df_matched_adjustments.empty and df_unmatched_adjustments.empty and df_unmatched_bank_records.empty:
        print("No data available for analysis. Please run reconciliation first.")
        return

    # 7.1 Reconciliation Overview
    print("\n### 7.1 Reconciliation Overview")
    reconciliation_status = pd.DataFrame({
        'Category': ['Matched Adjustments', 'Unmatched Adjustments', 'Unmatched Bank Records'],
        'Count': [len(df_matched_adjustments), len(df_unmatched_adjustments), len(df_unmatched_bank_records)]
    })
    print("\n**Counts of Matched/Unmatched Records:**")
    display(reconciliation_status)

    plt.figure(figsize=(8, 6))
    sns.barplot(x='Category', y='Count', data=reconciliation_status, palette='viridis')
    plt.title('Overview of Reconciliation Status')
    plt.ylabel('Number of Records')
    plt.xlabel('')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # 7.2 Unmatched Adjustments Analysis
    if not df_unmatched_adjustments.empty:
        print("\n### 7.2 Unmatched Adjustments Analysis")
        print("\n**Top Reasons for Unmatched Adjustments:**")
        reason_counts = df_unmatched_adjustments['Reason'].value_counts().reset_index()
        reason_counts.columns = ['Reason', 'Count']
        display(reason_counts)

        plt.figure(figsize=(10, 7))
        sns.barplot(x='Count', y='Reason', data=reason_counts, palette='magma')
        plt.title('Reasons for Unmatched Adjustments')
        plt.xlabel('Number of Adjustments')
        plt.ylabel('Reason')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.show()

        print("\n**Distribution of Unmatched Adjustment Amounts:**")
        plt.figure(figsize=(10, 6))
        sns.histplot(df_unmatched_adjustments['Amount'], bins=20, kde=True, color='red')
        plt.title('Distribution of Unmatched Adjustment Amounts')
        plt.xlabel('Amount')
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

    else:
        print("\n### 7.2 Unmatched Adjustments Analysis: No unmatched adjustments to analyze.")

    # 7.3 Unmatched Bank Records Analysis
    if not df_unmatched_bank_records.empty:
        print("\n### 7.3 Unmatched Bank Records Analysis")
        print("\n**Unmatched Bank Records by Bank/Table:**")
        bank_table_counts = df_unmatched_bank_records['Bank_Table'].value_counts().reset_index()
        bank_table_counts.columns = ['Bank_Table', 'Count']
        display(bank_table_counts)

        plt.figure(figsize=(10, 7))
        sns.barplot(x='Count', y='Bank_Table', data=bank_table_counts, palette='cividis')
        plt.title('Unmatched Bank Records by Bank Statement')
        plt.xlabel('Number of Records')
        plt.ylabel('Bank Statement')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.show()

        print("\n**Distribution of Unmatched Bank Record Amounts:**")
        plt.figure(figsize=(10, 6))
        sns.histplot(df_unmatched_bank_records['Amount'], bins=20, kde=True, color='blue')
        plt.title('Distribution of Unmatched Bank Record Amounts')
        plt.xlabel('Amount')
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

        # Time series analysis for unmatched bank records (if dates are present)
        df_unmatched_bank_records['_ParsedDate'] = pd.to_datetime(df_unmatched_bank_records['Date'])
        if not df_unmatched_bank_records['_ParsedDate'].empty:
            print("\n**Unmatched Bank Records Over Time:**")
            daily_unmatched = df_unmatched_bank_records.set_index('_ParsedDate').resample('D')['Amount'].count()
            if not daily_unmatched.empty:
                plt.figure(figsize=(12, 6))
                daily_unmatched.plot(kind='line', marker='o', linestyle='-', color='purple')
                plt.title('Daily Trend of Unmatched Bank Records')
                plt.xlabel('Date')
                plt.ylabel('Number of Unmatched Records')
                plt.grid(True)
                plt.tight_layout()
                plt.show()
            else:
                print("Not enough daily data to show time series trend for unmatched bank records.")

    else:
        print("\n### 7.3 Unmatched Bank Records Analysis: No unmatched bank records to analyze.")

    print("\n--- Data Analysis and Visualizations Complete ---")

# Button to trigger analysis and visualizations
analyze_btn = widgets.Button(description='Perform Analysis and Visualizations', button_style='success')
analysis_output = Output() # Dedicated output for analysis

def on_analyze_click(b):
    with analysis_output:
        clear_output()
        perform_data_analysis_and_visualizations()

analyze_btn.on_click(on_analyze_click)

# Display the button and output area for analysis
display(analyze_btn, analysis_output)
# perform_data_analysis_and_visualizations()


## 7. Data Science Analysis and Visualizations
This section provides a deeper dive into the reconciliation results using `pandas` for analysis and `seaborn` for visually appealing graphs.


Button(button_style='success', description='Perform Analysis and Visualizations', style=ButtonStyle())

Output()

In [54]:
# === Show results ===
print("\n===== SUMMARY =====")
print(f"✅ Adjustments Matched: {len(matched)}")
print(f"❌ Adjustments Unmatched: {len(unmatched)}")


===== SUMMARY =====
✅ Adjustments Matched: 0
❌ Adjustments Unmatched: 0


In [26]:
df_matched = pd.DataFrame(matched)
df_unmatched = pd.DataFrame(unmatched)
display(df_matched.head())
display(df_unmatched.head())

""


""
